<a href="https://colab.research.google.com/github/D10752002/emotion_detection_and_song_recommendation/blob/main/song_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [7]:
!unzip '/content/drive/MyDrive/face emotions dataset.zip' -d '/content'

Streaming output truncated to the last 5000 lines.
 extracting: /content/face emotions dataset/validation/angry/Training_81885304.jpg  
 extracting: /content/face emotions dataset/validation/angry/Training_81898650.jpg  
 extracting: /content/face emotions dataset/validation/angry/Training_81918581.jpg  
 extracting: /content/face emotions dataset/validation/angry/Training_81993292.jpg  
 extracting: /content/face emotions dataset/validation/angry/Training_82035674.jpg  
 extracting: /content/face emotions dataset/validation/angry/Training_82037343.jpg  
 extracting: /content/face emotions dataset/validation/angry/Training_82073038.jpg  
 extracting: /content/face emotions dataset/validation/angry/Training_82090617.jpg  
 extracting: /content/face emotions dataset/validation/angry/Training_82094076.jpg  
 extracting: /content/face emotions dataset/validation/angry/Training_82096123.jpg  
 extracting: /content/face emotions dataset/validation/angry/Training_82152293.jpg  
 extracting: /

In [9]:
PATH = '/content/face emotions dataset'

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

train_dataset = ImageDataGenerator(preprocessing_function= preprocess_input).flow_from_directory(train_dir,shuffle=True,target_size=(48,48),batch_size=32)
validation_dataset = ImageDataGenerator(preprocessing_function= preprocess_input).flow_from_directory(validation_dir,shuffle=True,target_size=(48,48),batch_size=32)

Found 20196 images belonging to 6 classes.
Found 5049 images belonging to 6 classes.


In [10]:
IMG_SHAPE = (48,48) + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
pred = Dense(6, activation='softmax')(x)

model = Model(base_model.input, pred)

9420800/9406464 [==============================] - 0s 0us/step


In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 24, 24, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 24, 24, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 24, 24, 32)   0           ['bn_Conv1[0][0]']               
                                                                                              

In [12]:
for layers in model.layers[:-5]:
  layers.trainable=False

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 24, 24, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 24, 24, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 24, 24, 32)   0           ['bn_Conv1[0][0]']               
                                                                                              

In [14]:
num_epochs = 20

optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(train_dataset, epochs=num_epochs, validation_data=validation_dataset)

Epoch 1/20
632/632 [==============================] - 58s 86ms/step - loss: 1.6126 - accuracy: 0.3385 - val_loss: 1.5246 - val_accuracy: 0.3860
Epoch 2/20
632/632 [==============================] - 51s 81ms/step - loss: 1.4433 - accuracy: 0.4319 - val_loss: 1.5014 - val_accuracy: 0.4005
Epoch 3/20
632/632 [==============================] - 51s 81ms/step - loss: 1.3221 - accuracy: 0.4901 - val_loss: 1.4795 - val_accuracy: 0.4161
Epoch 4/20
632/632 [==============================] - 51s 81ms/step - loss: 1.2014 - accuracy: 0.5539 - val_loss: 1.4821 - val_accuracy: 0.4187
Epoch 5/20
632/632 [==============================] - 51s 80ms/step - loss: 1.0734 - accuracy: 0.6057 - val_loss: 1.4827 - val_accuracy: 0.4272
Epoch 6/20
632/632 [==============================] - 51s 80ms/step - loss: 0.9371 - accuracy: 0.6705 - val_loss: 1.5483 - val_accuracy: 0.4203
Epoch 7/20
632/632 [==============================] - 52s 83ms/step - loss: 0.8030 - accuracy: 0.7277 - val_loss: 1.5933 - val_accuracy: